visjs (js, should be a python port/library)

In [3]:
# Import packages
import pyodbc
import json

In [4]:
# Load database configuration from JSON file
with open('config.json', 'r') as file:
    config = json.load(file)

In [8]:
# Connection parameters
sql_server_name = config['server_name']
sql_server_database_name = 'master' # Initialize as master, will be changed later on as we iterate through databases
sql_server_username = config['username']
sql_server_password = config['password']
sql_server_driver = config['driver']

In [9]:
"""
Connect to master database and extract all database names in the server
"""

# Connect to master database
conn_master = pyodbc.connect(f'DRIVER={sql_server_driver};SERVER={sql_server_name};DATABASE={sql_server_database_name};UID={sql_server_username};PWD={sql_server_password}')
cursor_master = conn_master.cursor()

# Query to get the list of all databases
cursor_master.execute("SELECT name FROM sys.databases")

# Fetch all query data
rows = cursor_master.fetchall()

# Initialize list to store database names
databases = []

# Add each row to list
for row in rows:
    databases.append(row.name)


# Close the master database cursor and connection
if cursor_master:
    cursor_master.close()
if conn_master:
    conn_master.close()

ProgrammingError: ('42000', '[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Reason: An instance-specific error occurred while establishing a connection to SQL Server. Connection was denied since Deny Public Network Access is set to Yes (https://docs.microsoft.com/azure/azure-sql/database/connectivity-settings#deny-public-network-access). To connect to this server, use the Private Endpoint from inside your virtual network (https://docs.microsoft.com/azure/sql-database/sql-database-private-endpoint-overview#how-to-set-up-private-link-for-azure-sql-database). (47073) (SQLDriverConnect)')

In [66]:
print(databases)

['master', 'recfi_data_analytics', 'recfi_data_test', 'recfi_data_analytics_test', 'recfi_data', 'recfi_data_dev', 'recfi_data_analytics_dev']


In [67]:
# Initialize a dictionary to store metadata
database_metadata = {}

# Iterate over each database
for db in databases:
    if db.lower() == 'master':
        continue
    
    # Connect to the database
    conn_db = pyodbc.connect(
        f'DRIVER={sql_server_driver};SERVER={sql_server_name};DATABASE={sql_server_database_name};UID={sql_server_username};PWD={sql_server_password}'
    )
    cursor_db = conn_db.cursor()
    
    # Query to get stored procedures, user tables, and views
    cursor_db.execute("""
        SELECT 
            o.name AS ObjectName,
            o.object_id AS ObjectID,
            o.type_desc AS ObjectType
        FROM 
            sys.objects AS o
        WHERE 
            o.type_desc IN ('SQL_STORED_PROCEDURE', 'USER_TABLE', 'VIEW')
    """)
    
    objects = cursor_db.fetchall()
    object_list = []
    
    for obj in objects:
        cursor_db.execute(f"""
            SELECT 
                definition
            FROM 
                sys.sql_modules
            WHERE 
                object_id = {obj.ObjectID}
        """)
        definition = cursor_db.fetchone()
        definition_text = definition.definition if definition else "No definition available"
        
        object_list.append({
            "ObjectName": obj.ObjectName,
            "ObjectID": obj.ObjectID,
            "ObjectType": obj.ObjectType,
            "Definition": definition_text
        })
    
    database_metadata[db] = object_list
    
    # Close the database cursor and connection
    if cursor_db:
        cursor_db.close()
    if conn_db:
        conn_db.close()

# Write metadata to a JSON file
with open('database_metadata.json', 'w', encoding='utf-8') as jsonfile:
    json.dump(database_metadata, jsonfile, ensure_ascii=False, indent=4)